In [34]:
import h5py
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model

In [35]:
directory = r'C:\Users\Brody\Documents\College Material\Fall 2022\BIOE 485\FinalProject\models\\'
model_name = r'unet_model_three_epochs_w_shuffle.h5'
model = load_model(directory + model_name)

In [36]:
@tf.function(input_signature=[tf.TensorSpec(None, tf.string)])
def process_path(file_path):
    # need to use non tf functions, call py_function
    image, mask = tf.py_function(func=process_path_helper, inp=[file_path], Tout=[tf.float32, tf.float32])
    image = tf.reshape(image, (240,240,4))
    mask = tf.reshape(mask, (240,240,3))
    return image, mask

def process_path_helper(file_path):
    # arbitrary Python code can be used here
    file_path = file_path.numpy().decode('ascii')
    f = h5py.File(file_path, 'r') 
    image = f['image'][()] 
    mask = f['mask'][()] 
    f.close()
    return image,mask

In [37]:
directory = r'C:\Users\Brody\Documents\College Material\Fall 2022\BIOE 485\FinalProject\BraTS2020_training_data\content\test\\'
ds_test = tf.data.Dataset.list_files(str(pathlib.Path(directory+'*.h5')))
batch_size = 1
ds_test = ds_test.map(process_path).batch(batch_size)

In [38]:
type(ds_test)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [39]:
num_tumor = 0
num_good = 0
num_tumor_correct = 0
num_good_correct = 0
num_false_tumor = 0
num_false_good = 0

count = 0
for batch in ds_test:
    if count % 500 == 0:
        print(count)

    image, mask = batch
    y = model(image).numpy()
    mask = mask.numpy()

    zeros = y < 0.5
    ones = y >= 0.5
    y[zeros] = 0.0
    y[ones] = 1.0
    
    num_tumor += np.sum(mask)
    num_good += np.sum(mask == 0.0)

    num_tumor_correct += np.sum(np.logical_and(y == 1.0, mask == 1.0))
    num_good_correct += np.sum(np.logical_and(y == 0.0, mask == 0.0))
    num_false_tumor += np.sum(np.logical_and(y == 1.0, mask == 0.0))
    num_false_good += np.sum(np.logical_and(y == 0.0, mask == 1.0))
        
    count += 1
    
print(num_tumor)
print(num_good)
print(num_tumor_correct)
print(num_good_correct)
print(num_false_tumor)
print(num_false_good)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
8814260.0
2080337740
4380743
2079089923
1247817
4433517


In [40]:
print(model_name)
print('PPV: ', str(num_tumor_correct/num_tumor))
print('NPV: ', str(num_good_correct/num_good))
print('Sensitivity: ', str(num_tumor_correct/(num_tumor_correct + num_false_good)))
print('Specificity: ', str(num_good_correct/(num_good_correct + num_false_tumor)))

unet_model_three_epochs_w_shuffle.h5
PPV:  0.49700632838150904
NPV:  0.9994001853756689
Sensitivity:  0.49700632838150904
Specificity:  0.9994001853756689


In [ ]:
loss = np.array([0.7215,0.6941,0.6934,0.6932,0.6932,0.6932,0.6932,0.6932,0.6931,0.6931])
epochs = range(1,11)
plt.figure()
plt.plot(epochs,loss)
plt.title('Loss vs Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Binary Cross Entropy Loss')

In [ ]:
import h5py
f = h5py.File(r'C:\Users\Brody\Documents\College Material\Fall 2022\BIOE 485\FinalProject\BraTS2020_training_data\content\test\volume_297_slice_80.h5', 'r') 
image = f['image'][()] 
mask = f['mask'][()] 
f.close()

image = tf.convert_to_tensor(image, dtype=tf.float32)
tf.ensure_shape(image,[240,240,4])
image = tf.expand_dims(image,0)
y = model.predict(image)
y = y[0,:,:,:]

zeros = y < 0.5
ones = y >= 0.5
y[zeros] = 0.0
y[ones] = 1.0

# Change layer to look at different mask channel
layer = 1
plt.figure()
plt.imshow(y[:,:,layer])
plt.figure()
plt.imshow(mask[:,:,layer])

In [21]:
import h5py
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [22]:
batch_size = 32
img_height = 240
img_width = 240

# Change to be directory of training set
directory = r'C:\Users\Brody\Documents\College Material\Fall 2022\BIOE 485\FinalProject\BraTS2020_training_data\content\data\\'
ds_train = tf.data.Dataset.list_files(str(pathlib.Path(directory+'*.h5')))

In [33]:
ds = ds_train.take(1).as_numpy_iterator()
for d in ds:
    print(d)

b'C:\\Users\\Brody\\Documents\\College Material\\Fall 2022\\BIOE 485\\FinalProject\\BraTS2020_training_data\\content\\data\\volume_132_slice_140.h5'
